### 세 파트 모두 연동한 코드

In [1]:
import os
import pickle
from dotenv import load_dotenv
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from openai import OpenAI

In [6]:
# ------------------------------------------
# 0. 환경 설정 및 API 초기화
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = None
try:
    client = OpenAI(api_key=OPENAI_API_KEY)
    print("OpenAI 클라이언트 초기화 완료")
except Exception as e:
    print("OpenAI 클라이언트 초기화 실패:", e)

# ------------------------------------------
# 1. MBTI 분류기 모델 로딩
model_paths = {
    "ei": ("models/ei_bilstm_model.h5", "tokenizers/ei_tokenizer.pkl"),
    "ns": ("models/ns_bilstm_model.h5", "tokenizers/ns_tokenizer.pkl"),
    "tf": ("models/tf_bilstm_model.h5", "tokenizers/tf_tokenizer.pkl"),
    "jp": ("models/jp_bilstm_model.h5", "tokenizers/jp_tokenizer.pkl"),
}

models, tokenizers = {}, {}
max_len = 300

for trait, (model_file, tokenizer_file) in model_paths.items():
    models[trait] = tf.keras.models.load_model(model_file)
    with open(tokenizer_file, "rb") as f:
        tokenizers[trait] = pickle.load(f)

def preprocess(text, tokenizer):
    seq = tokenizer.texts_to_sequences([text])
    return pad_sequences(seq, maxlen=max_len, padding='post')

def predict_mbti(text):
    result = ""
    label_pairs = {"ei": ("E", "I"), "ns": ("N", "S"), "tf": ("T", "F"), "jp": ("J", "P")}
    for trait in ["ei", "ns", "tf", "jp"]:
        x = preprocess(text, tokenizers[trait])
        pred = models[trait].predict(x)[0][0]
        label = label_pairs[trait][0] if round(pred) == 1 else label_pairs[trait][1]
        result += label
    return result

# ------------------------------------------
# 2. 사용자 대화 → MBTI 예측
print("\n안녕하세요! MBTI 예측 챗봇입니다.\n아래에 5개 문장을 입력해 주세요:")
session_inputs = []
for i in range(5):
    text = input(f"문장 {i+1}: ")
    session_inputs.append(text.strip())

combined_text = " ".join(session_inputs)
predicted_mbti = predict_mbti(combined_text)
print(f"\n[예측된 MBTI]: {predicted_mbti}")

# ------------------------------------------
# 3. 감정 분석 함수
def analyze_sentiment(user_text: str) -> str:
    if client is None:
        print("⚠ OpenAI 클라이언트 초기화 실패 → 감정 분석 불가")
        return "오류"

    allowed_emotions = [
        "기쁨","환희","유쾌","희열","만족","뿌듯함","감사","사랑","애정",
        "평온","안도","자신감","설렘","기대","존경","경외","흥분","열정","유머",
        "슬픔","우울","상실감","실망","좌절","후회","자책","외로움","분노","짜증",
        "혐오","경멸","두려움","공포","불안","초조","수치","부끄러움","피로","무기력","억울함",
        "혼란","갈등","당황","놀람","충격","호기심","의심","불신","그리움","향수",
        "중립"
    ]

    prompt = [
        {"role": "system", "content": f"너는 감정 분석가야. 다음 목록 중 하나의 감정만 반환해: {', '.join(allowed_emotions)}."},
        {"role": "user", "content": f"사용자 문장: '{user_text}'"}
    ]

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=prompt,
            temperature=0.0,
            max_tokens=5
        )
        sentiment = response.choices[0].message.content.strip()
        return sentiment if sentiment in allowed_emotions else "중립"
    except Exception as e:
        print(f"감정 분석 오류: {e}")
        return "중립"

# ------------------------------------------
# 4. GPT 상담 응답 생성기
MBTI_TONE = {
    "ENFP": "따뜻하고 유쾌하며 이모티콘을 자주 사용합니다.",
    "ISTJ": "분석적이고 신중하며 단정한 말투입니다.",
    "INFP": "섬세하고 감정에 공감하는 부드러운 말투입니다.",
    "ESTJ": "단호하고 체계적이며 사실 위주의 말투입니다.",
    "INTP": "논리적이고 중립적인 말투입니다.",
    "ESFJ": "친근하고 배려심 많은 말투로 위로를 잘 전합니다.",
    "ENTP": "재치 있고 유머러스하며 아이디어를 자유롭게 표현합니다.",
    "ISFJ": "조용하지만 따뜻하고 배려 깊은 말투로 상대를 존중합니다.",
    "INFJ": "직관적이며 깊이 있는 표현과 따뜻한 공감이 어우러진 말투입니다.",
    "ESTP": "직설적이고 에너지 넘치며 상황 중심적으로 조언합니다.",
    "ISFP": "차분하고 부드러우며 감정에 민감하게 반응합니다.",
    "INTJ": "간결하고 직관적인 말투이며 효율 중심적으로 접근합니다.",
    "ENTJ": "자신감 있고 목표 지향적이며 명확한 표현을 사용합니다.",
    "ENFJ": "따뜻하고 포용적인 말투로 감정에 깊이 공감합니다.",
    "ISTP": "과묵하고 실용적인 조언 위주로 핵심만 전달합니다.",
    "ESFP": "밝고 생동감 있는 말투로 친근하고 즉흥적인 표현을 자주 사용합니다."
}
MBTI_OPPOSITE_MAP = {
    "ISTJ": "ENFP", "ISFJ": "ENTP", "INFJ": "ESTP", "INTJ": "ESFP",
    "ISTP": "ENFJ", "ISFP": "ENTJ", "INFP": "ESTJ", "INTP": "ESFJ",
    "ESTP": "INFJ", "ESFP": "INTJ", "ENFP": "ISTJ", "ENTP": "ISFJ",
    "ESTJ": "INFP", "ESFJ": "INTP", "ENFJ": "ISTP", "ENTJ": "ISFP"
}

def get_tf_trait(mbti): return "F" if mbti[2] == "F" else "T"

def generate_response(user_input, mbti):
    tf_trait = get_tf_trait(mbti)
    tone = MBTI_TONE.get(MBTI_OPPOSITE_MAP.get(mbti, "ENFP"))
    instruction = "감정에 공감하며 부드럽게 위로해주세요." if tf_trait == "F" else "논리적으로 조언해주세요."
    system_msg = f"{MBTI_OPPOSITE_MAP.get(mbti)} 말투의 심리상담 챗봇입니다.\n말투: {tone}\n{instruction}"

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_input}
        ],
        max_tokens=300
    )
    return response.choices[0].message.content.strip()

# ------------------------------------------
# 5. 사용자 입력 및 분석 실행
user_input = input("\n고민을 입력하세요: ")

detected_emotion = analyze_sentiment(user_input)
print(f"\n[감정 분석 결과]: {detected_emotion}")

reply = generate_response(user_input, predicted_mbti)
print("\n[GPT 상담 응답]:")
print(reply)


OpenAI 클라이언트 초기화 완료



안녕하세요! MBTI 예측 챗봇입니다.
아래에 5개 문장을 입력해 주세요:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 603ms/step

[예측된 MBTI]: ISTP

[감정 분석 결과]: 슬픔

[GPT 상담 응답]:
안녕하세요, 힘든 상황에 처해 계시군요. 팀이 부진하다면 그에 대한 스트레스와 불안이 많이 쌓일 수 있습니다. 먼저, 상황을 분석하여 어떤 문제가 있는지 정확히 파악하는 것이 중요합니다. 팀 내 의사소통이 원활하지 않고 업무 분담이 불균형한지, 또는 목표 설정이 모호하거나 목표 달성을 위한 계획이 부족한지 등을 살펴보시는 것이 좋겠습니다. 그 후에 각 문제를 해결할 수 있는 방안을 찾아보고 팀원들과 솔직하게 소통하는 것도 좋은 방법일 것입니다. 함께 문제를 인식하고 해결책을 찾는 과정에서 팀원들의 동료애와 협력을 이끌어내는 리더십이 중요합니다. 힘들겠
